In [1]:
#!pip install --user --upgrade pip
#!pip install -U setuptools --user
#!pip install opencv-python --user
#!pip install os-sys --user
#!pip install face-recognition --user
#!pip install dlib --user
#!pip install keyboard --user

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import keyboard
import dlib
import face_recognition as face_recognition

In [3]:
users_folder = 'users'
users_subfolders = [ f.name for f in os.scandir(users_folder) if f.is_dir() ]

In [4]:
known_face_encodings = [] 
known_face_names = users_subfolders

In [5]:
for j in users_subfolders:
    
    subusers_path = users_folder + '/' + str(j) + '/'
    
    for i in os.listdir(subusers_path): 
        image = subusers_path + i
        image = face_recognition.load_image_file(image)
        image_encoding = face_recognition.face_encodings(image) 
        known_face_encodings.append(image_encoding[0])

In [6]:
current_face_locations = []
current_face_encodings = []
current_face_names = []
process_this_frame = True

In [7]:
cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    
    if process_this_frame:
        # Перетворюємо рамку з кольорів палітри BGR (бібліотеки OpenCV) на RGB кольори біблоіотеки face_recognition
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # знаходмо всі обличчя в даній раиці на відео
        current_face_locations = face_recognition.face_locations(rgb_frame)
        current_face_encodings = face_recognition.face_encodings(rgb_frame, current_face_locations)

        current_face_names = []
        
        for face in current_face_encodings:
            # перевіряємо чи подане облтччя є серед відомих користувачів
            matches = face_recognition.compare_faces(known_face_encodings, face)
            name = "Unknown user"
            color = (0, 0, 255) # обводимо червоним до розпізнавання

            # якщо поточний користувач є серед known_face_encodings, обираємо перше.
            if True in matches:
                
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            # або обираємо найближчого до цього користувача (за відстанню)
            face_distances = face_recognition.face_distance(known_face_encodings, face)
            best_match_index = np.argmin(face_distances)
            
            if matches[best_match_index]:                
                name = known_face_names[best_match_index]
                # обводимо зеленим
                color = (0, 255, 0)

            current_face_names.append(name)
    
    # завершуємо роботу з рамкою
    process_this_frame = not process_this_frame


    # результати
    for (top, right, bottom, left), name in zip(current_face_locations, current_face_names):

        # креслимо прямокутну рамку навколо обличчя
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

        # підписуємо рамку відповідно до результатів розпізнавання
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # лемонстрація отриманих результатів
    cv2.imshow('Face recognition...', frame)

    # Hit 'Esc' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('\x1b'):
        break

cap.release()
cv2.destroyAllWindows()